In [1]:
print("allo")

allo


In [2]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook, Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [3]:

sheetname='transaction_level'
filename = "qrm_market_value_and_exposure_derivatives_2021-12-31.xlsx"
filepath = "C:/Users/rubens/files/IDB/data/"  + filename

df_transactions = pd.read_excel(filepath, sheet_name=sheetname, skiprows=8)  


##### Data processing #####
# transactions with empty 'Security Identifier', replaced by 'transaction id'
df_transactions['Security Identifier'].fillna(df_transactions["Transaction ID"], inplace=True)

#copying allocation and index columns
df_transactions['df_allocation'] = df_transactions['Level 03.1']
df_transactions['df_index'] = df_transactions['Level 03.3']

# need to identify products that are in different columns
df_transactions['df_product'] = ''
not_aggregate_list = ['ABS MBS','Bond HTM','Bond HTM Accreting','Bond Trading','Bond Trading Accreting','Bond Trading Accreting Callable','Bond Trading Custom','CDs','CPs','Corporates','Deposit','EMP','Governments']

for index, row in df_transactions.iterrows():
    product_found = 'not found'
    
    if str(row['Level 08']) != 'nan' and row['Level 08'] not in not_aggregate_list:
        product_found = row['Level 08']
    elif  str(row['Level 07']) != 'nan' and row['Level 07'] not in not_aggregate_list:
         product_found = row['Level 07']
    elif  str(row['Level 06']) != 'nan' and row['Level 06'] not in not_aggregate_list:
        product_found = row['Level 06']
    elif  str(row['Level 05']) != 'nan' and row['Level 05'] not in not_aggregate_list:
        product_found = row['Level 05']
    else:
        product_found = 'product not found'

    df_transactions.at[index,'df_product'] = product_found
##### End of Data processing #####

list_allocations = df_transactions['df_allocation'].unique()
list_allocations = np.sort(list_allocations)
print(list_allocations)

list_products = df_transactions['df_product'].unique()
list_products = np.sort(list_products)
print(list_products)

list_indexes = df_transactions['df_index'].unique()
list_indexes = np.sort(list_indexes)
print(list_indexes)






###################################################################
# save transaction details
###################################################################
wb = Workbook()
ws = wb.active
#ws.showGridLines = False  ## not working ##

ws.title = "transaction_detail"
for r in dataframe_to_rows(df_transactions, index=False, header=True):
    ws.append(r)
###################################################################


###################################################################
# save unique values for allocation, product, index
###################################################################
ws2 = wb.create_sheet("indices", 0)

ws2.cell(column=1, row=1).value = "Allocations"
row_count = 2
for alloc in list_allocations:
    ws2.cell(column=1, row=row_count).value = alloc
    row_count += 1

ws2.cell(column=3, row=1).value = "Products"
row_count = 2
for product in list_products:
    ws2.cell(column=3, row=row_count).value = product
    row_count += 1

ws2.cell(column=5, row=1).value = "Indices"
row_count = 2
for index in list_indexes:
    ws2.cell(column=5, row=row_count).value = index
    row_count += 1

###################################################################


###################################################################
# save unique values for allocation, product, index
###################################################################
ws3 = wb.create_sheet("exposures", 0)

ws3.cell(column=1, row=1).value = "Key_identifier"
ws3.cell(column=2, row=1).value = "Allocation"
ws3.cell(column=3, row=1).value = "Product"
ws3.cell(column=4, row=1).value = "Index"
ws3.cell(column=5, row=1).value = "Face amount"
ws3.cell(column=6, row=1).value = "Book Value"
ws3.cell(column=7, row=1).value = "Market Value"
ws3.cell(column=8, row=1).value = "Economic Value"
ws3.cell(column=9, row=1).value = "BPV"
ws3.cell(column=10, row=1).value = "Convexity"

row_count = 2
for alloc in list_allocations:
    for product in list_products:
        for index in list_indexes:
            ws3.cell(column=1, row=row_count).value = alloc.replace(" ", "_") +  "." + product.replace(" ", "_") + "." + index.replace(" ", "_")
            ws3.cell(column=2, row=row_count).value = alloc
            ws3.cell(column=3, row=row_count).value = product
            ws3.cell(column=4, row=row_count).value = index

            row_count += 1
#print(type(list_allocations))
###################################################################

['CON (MOV)' 'CON (USD)' 'FCB (USD)' 'FFF (USD)' 'LCF (BRL)' 'LCF (COP)'
 'LCF (MXN)' 'LCF (USD)' 'LIQ (USD)' 'NSG (USD)' 'ORC (LOC)' 'OTH (USD)'
 'SCF (USD)' 'default unassigned']
['ALM Investments' 'Cash' 'Debt' 'Discount Notes' 'Duration_ISwap_P'
 'Duration_ISwap_R' 'Funding_AC_CSwap_P' 'Funding_AC_CSwap_R'
 'Funding_A_CSwap_P' 'Funding_A_CSwap_R' 'Funding_A_ISwap_P'
 'Funding_A_ISwap_R' 'Funding_CSwap_P' 'Funding_CSwap_R' 'Funding_ISwap_P'
 'Funding_ISwap_R' 'Investment CSwap_P' 'Investment CSwap_R'
 'Investment ISwap_P' 'Investment ISwap_R' 'Loan_CSwap_P' 'Loan_CSwap_R'
 'Loan_ISwap_P' 'Loan_ISwap_R' 'LoansD NSG' 'LoansD SG'
 'Swap Credit Value Adjustment']
['1M LIBOR' '3M LIBOR' '6M LIBOR' 'FIXED' 'NON_USD_FLT' 'SOFR'
 'default unassigned']


In [5]:
display(df_transactions)

,Security Identifier,Transaction ID,Level 02,Level 03,Level 04,Level 05,Level 06,Level 07,Level 08,Level 02.1,...,WAL,OAD,BPV,Convexity,MTM Spread,Spread BPV,DV01,df_allocation,df_index,df_product
0,114129OC,114129OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,0.030137,0.030555,-9.548240e+01,-0.049184,0.0,-9.231314e+01,-0.954824,LIQ (USD),FIXED,Investment ISwap_P
1,114129OC,114129OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_R,NaN,All,...,0.030137,0.030555,1.368665e+01,0.007050,0.0,1.323236e+01,0.136866,LIQ (USD),3M LIBOR,Investment ISwap_R
2,114376OC,114376OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,0.068493,0.069441,-7.811635e+02,-1.003306,0.0,-7.552602e+02,-7.811635,LIQ (USD),FIXED,Investment ISwap_P
3,114376OC,114376OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_R,NaN,All,...,0.068493,0.069441,8.356876e+01,0.107333,0.0,8.079763e+01,0.835688,LIQ (USD),3M LIBOR,Investment ISwap_R
4,114463OC,114463OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,0.087671,0.088882,-2.110770e+03,-3.535108,0.0,-2.047595e+03,-21.107698,LIQ (USD),FIXED,Investment ISwap_P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6654,ZS6149372,0000048737,Balance Sheet,Assets,Investments,Existing Investments,ALM Investments,Corporates,NaN,All,...,2.385246,2.388014,2.754067e+05,9073.801119,0.0,2.750349e+05,2754.066521,LIQ (USD),FIXED,ALM Investments
6655,ZS6149372,0000050040,Balance Sheet,Assets,Investments,Existing Investments,ALM Investments,Corporates,NaN,All,...,2.385246,2.388014,3.910774e+05,12884.797589,0.0,3.905495e+05,3910.774460,LIQ (USD),FIXED,ALM Investments
6656,ZS6212329,0000041276,Balance Sheet,Assets,Investments,Existing Investments,ALM Investments,Corporates,NaN,All,...,0.805479,0.813848,3.316876e+05,5366.585031,0.0,3.272132e+05,3316.876228,LIQ (USD),FIXED,ALM Investments
6657,ZS6663166,0000046591,Balance Sheet,Assets,Investments,Existing Investments,ALM Investments,Corporates,NaN,All,...,0.391781,0.172984,6.812818e+04,15361.947871,0.0,1.552202e+05,681.281757,LIQ (USD),NON_USD_FLT,ALM Investments


In [6]:
df_filtered = df_transactions[(df_transactions["df_allocation"]=="LIQ (USD)") & (df_transactions["df_index"]=='FIXED') & (df_transactions["df_product"]=='Investment ISwap_P')]

In [7]:
df_filtered

,Security Identifier,Transaction ID,Level 02,Level 03,Level 04,Level 05,Level 06,Level 07,Level 08,Level 02.1,...,WAL,OAD,BPV,Convexity,MTM Spread,Spread BPV,DV01,df_allocation,df_index,df_product
0,114129OC,114129OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,0.030137,0.030555,-95.482398,-0.049184,0.0,-92.313145,-0.954824,LIQ (USD),FIXED,Investment ISwap_P
2,114376OC,114376OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,0.068493,0.069441,-781.163485,-1.003306,0.0,-755.260233,-7.811635,LIQ (USD),FIXED,Investment ISwap_P
4,114463OC,114463OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,0.087671,0.088882,-2110.769813,-3.535108,0.0,-2047.594870,-21.107698,LIQ (USD),FIXED,Investment ISwap_P
6,114596OC,114596OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,0.106849,0.108319,-1299.669373,-2.792800,0.0,-1284.025726,-12.996694,LIQ (USD),FIXED,Investment ISwap_P
8,114958OC,114958OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,0.167123,0.169404,-3429.562544,-11.236301,0.0,-3477.744246,-34.295625,LIQ (USD),FIXED,Investment ISwap_P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,156122OC,156122OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,4.747945,2.531782,-139222.501474,-5452.821963,0.0,-137708.500028,-1392.225015,LIQ (USD),FIXED,Investment ISwap_P
976,156163OC,156163OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,3.057534,1.544423,-21841.730270,-630.178442,0.0,-21627.406776,-218.417303,LIQ (USD),FIXED,Investment ISwap_P
978,156217OC,156217OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,3.136986,1.624280,-98656.021925,-2910.427458,0.0,-97743.518651,-986.560219,LIQ (USD),FIXED,Investment ISwap_P
993,156978OC,156978OC,Balance Sheet,Assets,Investments,Existing Investments,ALM Inv Swaps,Investment ISwap_P,NaN,All,...,1.876712,1.125071,-6698.019018,-188.083402,0.0,-6643.233355,-66.980190,LIQ (USD),FIXED,Investment ISwap_P


In [8]:
df_filtered['BPV'].sum(axis = 0, skipna = True)


-5430229.657940654

In [9]:
filename = "qrm_market_value_and_exposure_derivatives_2021-12-31.xlsx"


In [15]:
date_str = filename[-15:-5]
date_str


'2021-12-31'